In [31]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import tqdm

from gensim.models import Word2Vec
from gensim.models.fasttext import FastText

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import *

import xgboost as xgb

In [32]:
train_df = pd.read_csv('/Users/nitanshjain/Documents/Projects/CASE/codefiles/subtask_1_data/train_subtask1_preprocessed_1.csv')
print(train_df.head())

               index                                               text  label
0     train_01_0_892  state alleged hacked sabata petros chale thirt...      1
1    train_01_1_2714  chale allegedly chased group thirty people hac...      0
2   train_01_10_2619     farmworkers strike resumed tuesday demands met      1
3  train_01_100_2680  demonstrators filed permit hold rally saturday...      1
4  train_01_101_3090  footage attack included pregnant woman hit pro...      1


In [33]:
dev_df = pd.read_csv('/Users/nitanshjain/Documents/Projects/CASE/codefiles/subtask_1_data/dev_subtask1_preprocessed_1.csv')
dev_df.head()

dev_df_labels = pd.read_csv('/Users/nitanshjain/Documents/Projects/CASE/tanfiona CausalNewsCorpus master data-V2/dev_subtask1.csv')
labels = dev_df_labels['label'].values
del(dev_df_labels)
print(labels)
print(dev_df.head())

[1 1 0 1 1 1 1 1 0 1 0 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 0 1 0 1 1 1 1 0 1 0
 0 1 1 1 0 1 1 0 0 1 0 0 0 1 1 0 1 1 0 1 0 1 1 0 0 0 1 0 0 0 0 1 1 1 1 0 0
 1 0 0 1 1 0 1 1 0 1 0 1 0 0 0 1 0 0 0 1 1 1 1 0 0 1 1 1 1 0 0 1 1 1 1 0 1
 0 1 0 1 1 0 1 0 1 1 1 0 1 0 0 1 1 1 0 0 0 1 1 1 1 1 1 0 1 1 1 0 0 0 1 0 1
 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 0 0 1 1 0 0 1 0 1 0 0 0 0 1 1 1 0 0 0
 0 1 1 1 0 1 1 0 1 0 0 1 1 0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 1 1 1 1 0 1 0 1 0
 0 1 1 1 0 0 0 1 1 1 1 1 0 0 1 0 1 0 0 0 0 1 0 1 1 0 1 0 0 1 1 1 0 0 0 1 1
 0 1 1 0 0 1 0 0 0 1 1 0 0 1 1 1 0 1 0 1 0 1 1 1 0 0 0 0 1 0 1 0 0 0 1 0 0
 1 1 1 0 1 1 0 0 1 0 1 0 0 0 1 0 0 1 1 1 1 1 1 0 0 1 0 1 0 1 0 0 1 1 1 1 0
 0 1 0 1 1 1 0]
               index                                               text
0    train_10_0_2136  movement catapulted headlines early august sem...
1     train_10_1_350  several thousand protesters took streets six p...
2   train_10_10_3104  protest saving medha life also preserving peop...
3  train_10_100_1188 

In [34]:

embedding_size = 150
window_size = 5
min_word = 5
down_sampling = 1e-2

fast_text_model_train = FastText(train_df.text.tolist(),
                      window=window_size,
                      min_count=min_word,
                      sample=down_sampling,
                      workers=4,
                      sg=1)

fast_text_model_dev = FastText(dev_df.text.tolist(),
                      window=window_size,
                      min_count=min_word,
                      sample=down_sampling,
                      workers=4,
                      sg=1)

fast_text_model_train.save("/Users/nitanshjain/Documents/Projects/CASE/codefiles/subtask1_embeddings/ft_model_train_text_1")
fast_text_model_train = Word2Vec.load("/Users/nitanshjain/Documents/Projects/CASE/codefiles/subtask1_embeddings/ft_model_train_text_1")

fast_text_model_dev.save("/Users/nitanshjain/Documents/Projects/CASE/codefiles/subtask1_embeddings/ft_model_dev_text_1")
fast_text_model_dev = Word2Vec.load("/Users/nitanshjain/Documents/Projects/CASE/codefiles/subtask1_embeddings/ft_model_dev_text_1")

In [35]:
def create_file(create_file, model_file, x):
    fast_text_model = Word2Vec.load(model_file)
    
    with open(create_file, 'w+') as word2vec_file:
        for index, row in x.iterrows():
            model_vector = (np.mean([fast_text_model.wv[token] for token in row['text']], axis=0)).tolist()
            if index == 0:
                header = ",".join(str(ele) for ele in range(150))
                word2vec_file.write(header)
                word2vec_file.write("\n")
            
            if type(model_vector) is list:
                line1 = ",".join( [str(vector_element) for vector_element in model_vector] )
            else:
                line1 = ",".join([str(0) for i in range(150)])
            word2vec_file.write(line1)
            word2vec_file.write('\n')
    
    df = pd.read_csv(create_file)
    return df

fast_text_train_filename = '/Users/nitanshjain/Documents/Projects/CASE/codefiles/subtask_1_data/train_ft_1.csv'
fast_text_train_model_file = '/Users/nitanshjain/Documents/Projects/CASE/codefiles/subtask1_embeddings/ft_model_train_text_1'
fast_text_train_embeddings_df = create_file(fast_text_train_filename, fast_text_train_model_file, train_df)

fast_text_train_embeddings = fast_text_train_embeddings_df.values

print(fast_text_train_embeddings_df.shape)
fast_text_train_embeddings_df.head()


fast_text_dev_filename = '/Users/nitanshjain/Documents/Projects/CASE/codefiles/subtask_1_data/dev_ft_1.csv'
fast_text_dev_model_file = '/Users/nitanshjain/Documents/Projects/CASE/codefiles/subtask1_embeddings/ft_model_dev_text_1'
fast_text_dev_embeddings_df = create_file(fast_text_dev_filename, fast_text_dev_model_file, dev_df)

fast_text_dev_embeddings = fast_text_dev_embeddings_df.values

print(fast_text_dev_embeddings_df.shape)
fast_text_dev_embeddings_df.head()

(3075, 150)
(340, 150)


,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
0,-0.028195,0.007591,0.035096,0.032376,0.021040,-0.035952,0.008841,0.098521,-0.058418,-0.045578,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.029227,0.008621,0.036354,0.033353,0.019263,-0.038063,0.010679,0.089704,-0.054878,-0.044369,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-0.029023,0.006393,0.032971,0.036659,0.024971,-0.038789,0.005379,0.090020,-0.059667,-0.041691,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-0.033026,0.009413,0.036592,0.027374,0.015940,-0.038907,0.012596,0.091721,-0.057417,-0.052107,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-0.029150,0.006021,0.034814,0.030217,0.020628,-0.040322,0.008536,0.100185,-0.060256,-0.045170,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
x = fast_text_train_embeddings_df.values
y = train_df['label'].values

In [40]:
xgb_model = xgb.XGBClassifier()

parameters = {
            'objective':['binary:logistic'],
            'learning_rate': [0.00001, 0.00005], 
            'max_depth': [6, 7, 8],
            'min_child_weight': [11],
            'subsample': [0.8],
            'colsample_bytree': [0.7],
            'n_estimators': [1000], #number of trees, change it to 1000 for better results
            'seed': [1337]
        }

clf = GridSearchCV(xgb_model, parameters, n_jobs=5, 
                   cv=StratifiedKFold(n_splits=3, random_state=42, shuffle=True), 
                   verbose=2, refit=True)

clf.fit(x, y)
print(clf.best_params_, clf.best_score_)

x_dev = fast_text_dev_embeddings_df.values
y_dev = labels

y_pred = clf.predict(x_dev)
print(classification_report(y_dev, y_pred))

Fitting 3 folds for each of 6 candidates, totalling 18 fits


In [ ]:
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
# 1 - no stopwords and removed few abbreviations
# 2 - with stopwords and removed few abbreviations
# 3 - no stopwords and didnt remove abbreviations
# 4 - with stopwords and didnt remove abbreviations